In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
from pinecone import Pinecone, ServerlessSpec
%pip install google-generativeai
import google.generativeai as genai
import json

Note: you may need to restart the kernel to use updated packages.


In [12]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
GOOGLE_API_KEY=os.environ.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
pc.create_index(
    name="rag",
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [13]:
import json
data = json.load(open("reviews.json"))

In [14]:
data = json.load(open("reviews.json"))

processed_data = []

for review in data["reviews"]:
    response = genai.embed_content(model="models/text-embedding-004", content=review['reviews'])
    embedding = response['embedding']
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["reviews"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

In [ ]:
processed_data[0]

In [15]:
index = pc.Index('rag')
index.upsert(
    vectors= processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [16]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}